# Profiling for Understanding

Profiling provides a complementary approach to step-through debugging for discovering what a seemingly (or not so) simple piece of code does. 

This example walks through the [SQLAlchemy ORM Quick Start](https://docs.sqlalchemy.org/en/20/orm/quickstart.html) to creating, populating, and querying a database with a domain model.

In [ ]:
from __future__ import annotations

import sys

if __name__ == "__main__" and "pyodide" in sys.modules:
    %pip install -r requirements.txt

## Demoing

Using the excellent [ipylab](https://github.com/jtpio/ipylab), this example will use a single `Pyinstrument` attached to the left side of the document with the 🔭 icon. 

In [ ]:
import pprint

from ipylab import Icon, JupyterFrontEnd, Panel

from ipyprofiler import Pyinstrument

In [ ]:
ps = Pyinstrument(interval=0.001)

In [ ]:
app = JupyterFrontEnd()
panel = locals()["panel"]
if panel:
    panel.close()
panel = Panel([ps.tabs(layout={"min_height": "60vh"})])
svgstr = (
    """<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">"""
    """<text x="4" y="18">🔭</text>"""
    """</svg>"""
)
panel.title.icon = Icon(name="ipyprofiler", svgstr=svgstr)
app.shell.add(panel, "right")
app.shell.expand_right()

## Importing

While `sqlalchemy` has few dependencies, it does a lot of work up-front. Note that re-running this cell will show an empty profile, as everything will already be loaded and cached.

In [ ]:
with ps.profile("importing"):
    from sqlalchemy import ForeignKey, String, create_engine, select
    from sqlalchemy.orm import (
        DeclarativeBase,
        Mapped,
        Session,
        mapped_column,
        relationship,
    )

## Modeling

Declaring a model is actually fairly involved, as modern `sqlalchemy` relies heavily on type annotations.

In [ ]:
with ps.profile("building a model"):

    class Base(DeclarativeBase):
        pass

    class User(Base):
        __tablename__ = "user_account"
        id: Mapped[int] = mapped_column(primary_key=True)
        name: Mapped[str] = mapped_column(String(30))
        fullname: Mapped[str | None]
        addresses: Mapped[list[Address]] = relationship(
            back_populates="user", cascade="all, delete-orphan"
        )

        def __repr__(self) -> str:
            return (
                f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"
            )

    class Address(Base):
        __tablename__ = "address"
        id: Mapped[int] = mapped_column(primary_key=True)
        email_address: Mapped[str]
        user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
        user: Mapped[User] = relationship(back_populates="addresses")

        def __repr__(self) -> str:
            return f"Address(id={self.id!r}, email_address={self.email_address!r})"

## Initializing

Using an in-memory `sqlite` database, we can see how much work goes into building a fresh database.

In [ ]:
with ps.profile("start your engine"):
    engine = create_engine("sqlite://")
    Base.metadata.create_all(engine)

## Building

Adding some instances shows the cost of executing that model.

In [ ]:
with ps.profile("build some instances"):
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")

## Saving

Having prepared all the work above, actually adding the instances is still a fair amount of work.

In [ ]:
with ps.profile("save instances"), Session(engine) as session:
    session.add_all([spongebob, sandy, patrick])
    session.commit()

## Querying

With the data in the database, we can now see what goes into querying for some instances.

In [ ]:
with ps.profile("query instances"), Session(engine) as session:
    result = session.execute(select(User).order_by(User.id))
    pprint.pprint(result.all())